#### 분류(classification) 

In [1]:
# KNN

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 

In [2]:
# titanic 
df = sns.load_dataset('titanic')

In [3]:
df.head(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True


In [4]:
#  IPython 디스플레이 설정 - 출력할 열의 개수 한도 늘리기
pd.set_option('display.max_columns', 15)

### EDA(탐색적 데이터 분석)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [6]:
df.embarked[:10]

0    S
1    C
2    S
3    S
4    S
5    Q
6    S
7    S
8    S
9    C
Name: embarked, dtype: object

In [7]:
df.embark_town[:10]

0    Southampton
1      Cherbourg
2    Southampton
3    Southampton
4    Southampton
5     Queenstown
6    Southampton
7    Southampton
8    Southampton
9      Cherbourg
Name: embark_town, dtype: object

In [8]:
# NaN 값이 많은 deck 열 삭제
# embarked 내용이 겹치는 embark_town 열 삭제 

df.drop(['deck','embark_town'], axis=1)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,yes,True


In [9]:
ndf = df.drop(['deck','embark_town'], axis=1)
ndf.head(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,yes,True


In [10]:
ndf.columns

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'alive', 'alone'],
      dtype='object')

In [11]:
ndf.columns.values

array(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'alive', 'alone'],
      dtype=object)

In [12]:
ndf.age[:10]

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: age, dtype: float64

In [13]:
# age 열에 나이 데이터가 없는 모든 행을 삭제
ndf.isna().sum() 
# 전체 데이터 891개 중 age 열에 NaN 값이 177개 발견 >> 삭제하기로 함 

survived        0
pclass          0
sex             0
age           177
sibsp           0
parch           0
fare            0
embarked        2
class           0
who             0
adult_male      0
alive           0
alone           0
dtype: int64

In [14]:
ndf.dropna(subset=['age'], how='any', axis=0)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
885,0,3,female,39.0,0,5,29.1250,Q,Third,woman,False,no,False
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,yes,True
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,yes,True


In [15]:
ndf = ndf.dropna(subset=['age'], how='any', axis=0)
ndf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 714 entries, 0 to 890
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   survived    714 non-null    int64   
 1   pclass      714 non-null    int64   
 2   sex         714 non-null    object  
 3   age         714 non-null    float64 
 4   sibsp       714 non-null    int64   
 5   parch       714 non-null    int64   
 6   fare        714 non-null    float64 
 7   embarked    712 non-null    object  
 8   class       714 non-null    category
 9   who         714 non-null    object  
 10  adult_male  714 non-null    bool    
 11  alive       714 non-null    object  
 12  alone       714 non-null    bool    
dtypes: bool(2), category(1), float64(2), int64(4), object(4)
memory usage: 63.6+ KB


In [16]:
# embarked 어떻게 하지?
# object(문자열) 
# NaN이 전체의 02개네? >> 삭제 말고 채우는 걸로 가자. 
# 어떻게? 최빈값 (가장 많이 승객이 탑승한 도시로 결정)

ndf.embarked.value_counts()

S    554
C    130
Q     28
Name: embarked, dtype: int64

In [17]:
ndf.embarked.value_counts(dropna=True)

S    554
C    130
Q     28
Name: embarked, dtype: int64

In [18]:
ndf.embarked.value_counts(dropna=True).idxmax()

'S'

In [19]:
most_freq = ndf.embarked.value_counts(dropna=True).idxmax()

In [20]:
ndf.describe() # 수치형 데이터만 출력

,survived,pclass,age,sibsp,parch,fare
count,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000
mean,0.406162,2.236695,29.699118,0.512605,0.431373,34.694514
std,0.491460,0.838250,14.526497,0.929783,0.853289,52.918930
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,20.125000,0.000000,0.000000,8.050000
50%,0.000000,2.000000,28.000000,0.000000,0.000000,15.741700
75%,1.000000,3.000000,38.000000,1.000000,1.000000,33.375000
max,1.000000,3.000000,80.000000,5.000000,6.000000,512.329200


In [21]:
ndf.describe(include='all')

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,alive,alone
count,714.000000,714.000000,714,714.000000,714.000000,714.000000,714.000000,712,714,714,714,714,714
unique,NaN,NaN,2,NaN,NaN,NaN,NaN,3,3,3,2,2,2
top,NaN,NaN,male,NaN,NaN,NaN,NaN,S,Third,man,True,no,True
freq,NaN,NaN,453,NaN,NaN,NaN,NaN,554,355,413,413,424,404
mean,0.406162,2.236695,NaN,29.699118,0.512605,0.431373,34.694514,NaN,NaN,NaN,NaN,NaN,NaN
std,0.491460,0.838250,NaN,14.526497,0.929783,0.853289,52.918930,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,1.000000,NaN,0.420000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,0.000000,1.000000,NaN,20.125000,0.000000,0.000000,8.050000,NaN,NaN,NaN,NaN,NaN,NaN
50%,0.000000,2.000000,NaN,28.000000,0.000000,0.000000,15.741700,NaN,NaN,NaN,NaN,NaN,NaN
75%,1.000000,3.000000,NaN,38.000000,1.000000,1.000000,33.375000,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
ndf.embarked.fillna(most_freq, inplace=True)

In [23]:
ndf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 714 entries, 0 to 890
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   survived    714 non-null    int64   
 1   pclass      714 non-null    int64   
 2   sex         714 non-null    object  
 3   age         714 non-null    float64 
 4   sibsp       714 non-null    int64   
 5   parch       714 non-null    int64   
 6   fare        714 non-null    float64 
 7   embarked    714 non-null    object  
 8   class       714 non-null    category
 9   who         714 non-null    object  
 10  adult_male  714 non-null    bool    
 11  alive       714 non-null    object  
 12  alone       714 non-null    bool    
dtypes: bool(2), category(1), float64(2), int64(4), object(4)
memory usage: 63.6+ KB


In [24]:
ndf.head(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,yes,True


In [25]:
ndf = ndf[['survived', 'pclass', 'sex','age', 'sibsp', 'parch', 'embarked']]

In [26]:
ndf.head(3)

,survived,pclass,sex,age,sibsp,parch,embarked
0,0,3,male,22.0,1,0,S
1,1,1,female,38.0,1,0,C
2,1,3,female,26.0,0,0,S


In [27]:
# ndf 의 sex가 문자열로 구성되어 있음 >> 라벨 작업 필요 

pd.get_dummies(ndf.sex)

,female,male
0,0,1
1,1,0
2,1,0
3,1,0
4,0,1
...,...,...
885,1,0
886,0,1
887,1,0
889,0,1


In [28]:
gender = pd.get_dummies(ndf.sex)

In [29]:
gender

,female,male
0,0,1
1,1,0
2,1,0
3,1,0
4,0,1
...,...,...
885,1,0
886,0,1
887,1,0
889,0,1


In [30]:
pd.concat([ndf, gender], axis=1)

,survived,pclass,sex,age,sibsp,parch,embarked,female,male
0,0,3,male,22.0,1,0,S,0,1
1,1,1,female,38.0,1,0,C,1,0
2,1,3,female,26.0,0,0,S,1,0
3,1,1,female,35.0,1,0,S,1,0
4,0,3,male,35.0,0,0,S,0,1
...,...,...,...,...,...,...,...,...,...
885,0,3,female,39.0,0,5,Q,1,0
886,0,2,male,27.0,0,0,S,0,1
887,1,1,female,19.0,0,0,S,1,0
889,1,1,male,26.0,0,0,C,0,1


In [31]:
ndf = pd.concat([ndf, gender], axis=1)

In [32]:
pd.get_dummies(ndf.embarked)

,C,Q,S
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1
...,...,...,...
885,0,1,0
886,0,0,1
887,0,0,1
889,1,0,0


In [33]:
pd.get_dummies(ndf.embarked, prefix='town')

,town_C,town_Q,town_S
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1
...,...,...,...
885,0,1,0
886,0,0,1
887,0,0,1
889,1,0,0


In [34]:
oh_embarked = pd.get_dummies(ndf.embarked, prefix='town')

In [35]:
pd.concat([ndf, oh_embarked], axis=1)

,survived,pclass,sex,age,sibsp,parch,embarked,female,male,town_C,town_Q,town_S
0,0,3,male,22.0,1,0,S,0,1,0,0,1
1,1,1,female,38.0,1,0,C,1,0,1,0,0
2,1,3,female,26.0,0,0,S,1,0,0,0,1
3,1,1,female,35.0,1,0,S,1,0,0,0,1
4,0,3,male,35.0,0,0,S,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
885,0,3,female,39.0,0,5,Q,1,0,0,1,0
886,0,2,male,27.0,0,0,S,0,1,0,0,1
887,1,1,female,19.0,0,0,S,1,0,0,0,1
889,1,1,male,26.0,0,0,C,0,1,1,0,0


In [36]:
ndf = pd.concat([ndf, oh_embarked], axis=1)

In [37]:
ndf.head(3)

,survived,pclass,sex,age,sibsp,parch,embarked,female,male,town_C,town_Q,town_S
0,0,3,male,22.0,1,0,S,0,1,0,0,1
1,1,1,female,38.0,1,0,C,1,0,1,0,0
2,1,3,female,26.0,0,0,S,1,0,0,0,1


In [38]:
ndf.drop(['sex','embarked'], axis=1, inplace=True)
ndf.head(3)

,survived,pclass,age,sibsp,parch,female,male,town_C,town_Q,town_S
0,0,3,22.0,1,0,0,1,0,0,1
1,1,1,38.0,1,0,1,0,1,0,0
2,1,3,26.0,0,0,1,0,0,0,1


In [39]:
ndf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 714 entries, 0 to 890
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   survived  714 non-null    int64  
 1   pclass    714 non-null    int64  
 2   age       714 non-null    float64
 3   sibsp     714 non-null    int64  
 4   parch     714 non-null    int64  
 5   female    714 non-null    uint8  
 6   male      714 non-null    uint8  
 7   town_C    714 non-null    uint8  
 8   town_Q    714 non-null    uint8  
 9   town_S    714 non-null    uint8  
dtypes: float64(1), int64(4), uint8(5)
memory usage: 37.0 KB


In [40]:
# 데이터 셋 분리 (훈련용, 테스트용)

# 속성(변수) 선택 
# survived pclass age sibsp parch female male town_C town_Q town_S

x = ndf[['pclass', 'age', 'sibsp', 'parch', 'female', 'male', 'town_C', 'town_Q', 'town_S']]
y = ndf['survived']
x, y

(     pclass   age  sibsp  parch  female  male  town_C  town_Q  town_S
 0         3  22.0      1      0       0     1       0       0       1
 1         1  38.0      1      0       1     0       1       0       0
 2         3  26.0      0      0       1     0       0       0       1
 3         1  35.0      1      0       1     0       0       0       1
 4         3  35.0      0      0       0     1       0       0       1
 ..      ...   ...    ...    ...     ...   ...     ...     ...     ...
 885       3  39.0      0      5       1     0       0       1       0
 886       2  27.0      0      0       0     1       0       0       1
 887       1  19.0      0      0       1     0       0       0       1
 889       1  26.0      0      0       0     1       1       0       0
 890       3  32.0      0      0       0     1       0       1       0
 
 [714 rows x 9 columns],
 0      0
 1      1
 2      1
 3      1
 4      0
       ..
 885    0
 886    0
 887    1
 889    1
 890    0
 Name: surv

In [48]:
from sklearn.preprocessing import StandardScaler

StandardScaler().fit_transform(x)

array([[ 0.91123237, -0.53037664,  0.52457013, ..., -0.47180795,
        -0.20203051,  0.53307848],
       [-1.47636364,  0.57183099,  0.52457013, ...,  2.11950647,
        -0.20203051, -1.87589641],
       [ 0.91123237, -0.25482473, -0.55170307, ..., -0.47180795,
        -0.20203051,  0.53307848],
       ...,
       [-1.47636364, -0.73704057, -0.55170307, ..., -0.47180795,
        -0.20203051,  0.53307848],
       [-1.47636364, -0.25482473, -0.55170307, ...,  2.11950647,
        -0.20203051, -1.87589641],
       [ 0.91123237,  0.15850313, -0.55170307, ..., -0.47180795,
         4.94974747, -1.87589641]])

In [49]:
# 정규화 

# 단위가 틀리네? >> 정규화 과정 처리 

from sklearn.preprocessing import StandardScaler

sc_x = StandardScaler()

In [50]:
sc_x.fit(x)

StandardScaler()

In [51]:
sc_x.fit(x).transform(x)

array([[ 0.91123237, -0.53037664,  0.52457013, ..., -0.47180795,
        -0.20203051,  0.53307848],
       [-1.47636364,  0.57183099,  0.52457013, ...,  2.11950647,
        -0.20203051, -1.87589641],
       [ 0.91123237, -0.25482473, -0.55170307, ..., -0.47180795,
        -0.20203051,  0.53307848],
       ...,
       [-1.47636364, -0.73704057, -0.55170307, ..., -0.47180795,
        -0.20203051,  0.53307848],
       [-1.47636364, -0.25482473, -0.55170307, ...,  2.11950647,
        -0.20203051, -1.87589641],
       [ 0.91123237,  0.15850313, -0.55170307, ..., -0.47180795,
         4.94974747, -1.87589641]])

In [52]:
sc_x = sc_x.fit(x).transform(x)

In [53]:
# 데이터 분리 (훈련용, 테스트용, 7:3 비율) 

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test =\
train_test_split(sc_x, y, test_size=0.3, random_state=42)

In [54]:
x_train, x_train.shape

(array([[ 0.91123237, -0.59926462, -0.55170307, ..., -0.47180795,
         -0.20203051,  0.53307848],
        [-0.28256564, -1.70147225,  0.52457013, ..., -0.47180795,
         -0.20203051,  0.53307848],
        [ 0.91123237, -0.08260479, -0.55170307, ...,  2.11950647,
         -0.20203051, -1.87589641],
        ...,
        [-1.47636364,  0.77849493, -0.55170307, ...,  2.11950647,
         -0.20203051, -1.87589641],
        [ 0.91123237,  0.22739111,  0.52457013, ..., -0.47180795,
         -0.20203051,  0.53307848],
        [ 0.91123237,  0.22739111, -0.55170307, ...,  2.11950647,
         -0.20203051, -1.87589641]]),
 (499, 9))

In [55]:
x_test, x_test.shape

(array([[-0.28256564,  0.8473829 , -0.55170307, ..., -0.47180795,
         -0.20203051,  0.53307848],
        [-0.28256564, -1.83924821,  0.52457013, ..., -0.47180795,
         -0.20203051,  0.53307848],
        [-0.28256564, -0.0481608 ,  0.52457013, ..., -0.47180795,
         -0.20203051,  0.53307848],
        ...,
        [ 0.91123237, -1.90813618, -0.55170307, ..., -0.47180795,
         -0.20203051,  0.53307848],
        [ 0.91123237, -1.5636963 ,  3.75338974, ..., -0.47180795,
          4.94974747, -1.87589641],
        [-1.47636364,  1.81181458, -0.55170307, ...,  2.11950647,
         -0.20203051, -1.87589641]]),
 (215, 9))

모델링

In [56]:
# knn 분류 모형 

from sklearn.neighbors import KNeighborsClassifier

In [58]:
# K= 5 설정 

knn = KNeighborsClassifier(n_neighbors=5)

In [59]:
# 훈련 (train data)

knn.fit(x_train, y_train)

KNeighborsClassifier()

In [60]:
# test data로 y_hat 만들어 예측(분류)

knn.predict(x_test) # 반드시, x_test 데이터 입력 

array([0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1,
       0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0], dtype=int64)

In [61]:
y_hat = knn.predict(x_test)  

In [62]:
# 모형 성능 평가 

from sklearn.metrics import *

In [63]:
confusion_matrix(y_test, y_hat)  # 실제 라벨된 값, 예측한 값 

array([[109,  17],
       [ 27,  62]], dtype=int64)

In [66]:
# 평가 지표 

print(classification_report(y_test, y_hat))

              precision    recall  f1-score   support

           0       0.80      0.87      0.83       126
           1       0.78      0.70      0.74        89

    accuracy                           0.80       215
   macro avg       0.79      0.78      0.79       215
weighted avg       0.79      0.80      0.79       215



In [ ]:
# 결과해석 
# acc (정확도) :  0.80 (80%의 정확도)
# f1-score : 미생존자(0) : 0.83, 생존자(1): 0.74 
# 예측 정확도가 차이가 발생함 